In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import UnstructuredURLLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.question_answering import load_qa_chain
import warnings
warnings.filterwarnings('ignore')
llm = ChatGroq(
    model_name = "llama-3.3-70b-versatile",
    temperature = 0.25,
    groq_api_key = "api_key"
)

In [3]:
from langchain_community.document_loaders import UnstructuredURLLoader

urls = [
    "https://google.github.io/styleguide/docguide/style.html",
    "https://www.markdownguide.org/basic-syntax/",
    "https://gist.github.com/rt2zz/e0a1d6ab2682d2c47746950b84c0b6ee",
    "https://gist.github.com/allysonsilva/85fff14a22bbdf55485be947566cc09e",
    "https://israelmitolu.hashnode.dev/markdown-for-technical-writers-tips-tricks-and-best-practices"
]

loader = UnstructuredURLLoader(urls=urls)

guideline = loader.load()

In [4]:
import os
from langchain.document_loaders import DirectoryLoader
from langchain import PromptTemplate, LLMChain

loader = DirectoryLoader('../Cover Letter Generator', glob="**/*.py")  # Load all Python files
template_md = DirectoryLoader("./", glob = "**/*.md")
documents = loader.load()
template_md_document = template_md.load()

# Combine the text from all loaded documents
code_content = "\n\n".join([doc.page_content for doc in documents])
template_content = "\n\n".join([doc.page_content for doc in template_md_document])

In [5]:
import yaml
def load_openapi_spec(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)
    
api = load_openapi_spec("openapi.yaml")

In [6]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") 

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
documents = guideline
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50 
)

chunked_texts = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    chunked_texts.extend(chunks)

vectorstore = FAISS.from_texts(chunked_texts, embedding_model)

In [8]:
retriever = vectorstore.as_retriever()

In [9]:
prompt_template = PromptTemplate(
    input_variables=["context", "question", 'code', "api"],
    template="""
    You are a Markdown expert who only take a code and description and then write Markdown file in md format not more detail with the context guidelines that were provided.

    Context:
    {context}
    Here's the Markdown text you need to review
    Question:
    {question}
    And this is the code of the project :
    {code}
    And the OpenAPI specs:
    {api}
    Answer: 
    Please only write a result in markdown format. Your result should be clear, concise, and actionable.
    """ 
)

In [10]:
combine_docs_chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt_template)  
# for longer context map_reduce can be used
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)

In [11]:
retrieval_qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    combine_docs_chain=combine_docs_chain,  
    question_generator=question_generator,  
    return_source_documents=True
)

In [12]:
chat_history = [] 
question  = """
This is a project about Language model customization to customize LLama 3.1 for generating cover letter by given job description and then it will convert it into latex file and then give pdf output

"""
result = retrieval_qa_chain({"question": question, 'code': code_content,"api": api, "chat_history": chat_history})

In [1]:
print("Answer:", result['answer'])
# print("Source Documents:", result['source_documents'])

# Language Model Customization for Cover Letter Generation
===========================================================

## Overview
-----------

This project aims to customize the LLaMA 3.1 language model to generate cover letters based on given job descriptions. The model will convert the generated cover letter into a LaTeX file and produce a PDF output.

## Requirements
------------

* Python 3.x
* Streamlit
* Pylatex
* Langchain
* Groq API Key

## Code
-----

### Importing Libraries

```python
import streamlit as st
import subprocess
from pylatex.utils import NoEscape
from pylatex import Document
import re
from langchain_groq import ChatGroq
from langchain.schema import Document
from langchain_core.prompts import PromptTemplate
```

### Initializing the LLM

```python
llm = ChatGroq(
    model_name="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key="api_key"
)
```

### Defining the Streamlit App

```python
st.title('Cover Letter Generator')
job_description = st.text_area("Paste the job description here:", "")
```

### Generating the Cover Letter

```python
if st.button('Generate Cover Letter'):
    if job_description:
        try:
            # Create Document and invoke LLM
            doc = Document(page_content=job_description)
            prompt_extract = PromptTemplate.from_template(
                # ... (prompt template)
            )
            chain = prompt_extract | llm
            res = chain.invoke(input={"doc": doc.page_content, "cover_latter": cover_letter_template})
            latex_code = res.content
            latex_code = latex_code.replace("\\\\", '\\')
            latex_code = latex_code.replace("\\n", "\n")

            # Create LaTeX file and generate PDF
            latex_codes = r""" \documentclass{letter} ... """
            create_pdf_from_latex(latex_codes, "cover_letter")
            st.success('PDF generated successfully! And you can check the main cover_letter.pdf file in the root folder.')
        except Exception as e:
            st.error(f"An error occurred: {e}")
    else:
        st.error('Please paste a job description.')
```

## Usage
-----

1. Run the Streamlit app by executing `streamlit run app.py` in your terminal.
2. Paste the job description in the text area.
3. Click the "Generate Cover Letter" button.
4. The app will generate a PDF file named "cover_letter.pdf" in the root folder.

## Notes
-----

* Make sure to replace the `groq_api_key` with your actual Groq API key.
* The `cover_letter_template` is a static template and may need to be modified to fit your specific needs.
* The `prompt_extract` template is used to extract the relevant information from the job description and cover letter template.

In [18]:
prompt_template = PromptTemplate(
    input_variables=["context", "question", "code", "api", "chunk_instruction"],
    template="""
    You are a Markdown expert. Use the provided context, project description, code, and API specifications to create a well-structured Markdown file. 
    Focus on summarizing and explaining each part in a way that’s accessible and useful.

    **Guidelines**:
    - Use clear headings and subheadings.
    - Explain the purpose of each section (e.g., Overview, Setup Instructions, Key Functions, API Usage).
    - Only summarize code and API specifications briefly; focus on their purpose and functionality.
    - Include examples or sample usage where relevant.
    
    ## Project Context
    {context}

    ## Project Description
    {question}

    ## Code Overview
    - Summarize the key parts of the code, providing descriptions for each.
    - Example usage or relevant snippets may be included if helpful.

    ## API Specifications
    {api}
    - Summarize the API specifications focusing on key endpoints and parameters.

    ## Instructions
    {chunk_instruction}

    ### Answer (Markdown Chunk):
    """
)


def generate_markdown_in_chunks(question, code, chunk_size=2000):
    chat_history = []

    current_markdown = ""
    chunk_instruction = "Generate the beginning of the Markdown file (up to ~2000 characters)."
    max_attempts = 2


    for attempt in range(max_attempts):
        result = retrieval_qa_chain({
            "question": question,
            "code": code,
            "api": api,
            "chat_history": chat_history,
            "chunk_instruction": chunk_instruction
        })

        chunk = result['answer']
        current_markdown += chunk

        if "<!-- END_OF_MARKDOWN -->" in chunk:
            break 

        if attempt == max_attempts - 1:
            chunk += "<!-- END_OF_MARKDOWN -->" 
            break

        if len(current_markdown) < chunk_size - 500:
            chunk_instruction = f"Continue generating the Markdown from where you left off (aim for around {chunk_size - len(current_markdown)} characters, ending at a logical break if possible)."
        else:
            chunk_instruction = "Conclude the current section of the Markdown and add an end-of-markdown marker if the entire Markdown file is complete. Otherwise, continue to the next section ensuring the chunk ends at a logical breaking point."
        yield chunk
        chat_history.append((question, chunk))



question  = """
This is a project about Language model customization to customize LLama 3.1 for generating cover letter by given job description and then it will convert it into latex file and then give pdf output

"""
for markdown_chunk in generate_markdown_in_chunks(question, code_content):
    print(markdown_chunk)
print("Markdown generation complete.")

# Language Model Customization for Cover Letter Generation

## Overview
-----------

This project aims to customize the LLaMA 3.1 language model to generate cover letters based on a given job description. The model will convert the generated cover letter into a LaTeX file and produce a PDF output.

## Requirements
------------

* Python 3.x
* Streamlit
* Pylatex
* Langchain
* Groq API key

## Installation
------------

```bash
pip install streamlit pylatex langchain
```

## Usage
-----

1. Clone the repository and navigate to the project directory.
2. Install the required dependencies using `pip install -r requirements.txt`.
3. Run the Streamlit app using `streamlit run app.py`.
4. Paste the job description into the text area.
5. Click the "Generate Cover Letter" button to generate the cover letter.
6. The generated cover letter will be displayed in the text area, and a PDF file will be saved in the root directory.

## Code
-----

### Importing Libraries

```python
import streamlit as 

In [19]:
from IPython.display import Markdown

for markdown_chunk in generate_markdown_in_chunks(question, code_content):
    display(Markdown(markdown_chunk))
#     display(df.nunique())
#     print(markdown_chunk)

# Language Model Customization for Cover Letter Generation
===========================================================

## Overview
-----------

This project aims to customize the LLaMA 3.1 language model to generate cover letters based on a given job description. The model will convert the generated cover letter into a LaTeX file and produce a PDF output.

## Requirements
------------

* Python 3.x
* Streamlit
* Pylatex
* Langchain
* Groq API key

## Installation
------------

```bash
pip install streamlit pylatex langchain
```

## Usage
-----

1. Clone the repository and navigate to the project directory.
2. Install the required dependencies using `pip install -r requirements.txt`.
3. Run the Streamlit app using `streamlit run app.py`.
4. Paste the job description into the text area.
5. Click the "Generate Cover Letter" button to generate the cover letter.
6. The generated cover letter will be displayed in the text area, and a PDF file will be saved in the root directory.

## Code
-----

### Importing Libraries

```python
import streamlit as st
import subprocess
from pylatex.utils import NoEscape
from pylatex import Document
import re
from langchain_groq import ChatGroq
from langchain.schema import Document
from langchain_core.prompts import PromptTemplate
```

### Initializing the LLM

```python
llm = ChatGroq(
    model_name="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key="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"
)
```

### Defining the Streamlit App

```python
st.title('Cover Letter Generator')
job_description = st.text_area("Paste the job description here:", "")
```

### Generating the Cover Letter

```python
if st.button('Generate Cover Letter'):
    if job_description:
        try:
            # Create Document and invoke LLM
            doc = Document(page_content=job_description)
            prompt_extract = PromptTemplate.from_template(
                # ... (template code)
            )
            chain = prompt_extract | llm
            res = chain.invoke(input={"doc": doc.page_content, "cover_latter": cover_letter_template})
            latex_code = res.content
            latex_code = latex_code.replace("\\\\", '\\')
            latex_code = latex_code.replace("\\n", "\n")

            # Create PDF from LaTeX code
            latex_codes = r""" \documentclass{letter} \usepackage{hyperref} \usepackage[margin=1in, top=1in]{geometry} % Adjust margins here

            \signature{Amirmahdi Aboutalebi} \address{Genova, Italy \\ +393513211513 \\ \href{mailto:amir.abootalebi2001@gmail.com}{amir.abootalebi2001@gmail.com} \\ \href{https://amirmahdiabtl.github.io/}{amirmahdiabtl.github.io}}

            \begin{document}

            \begin{letter}{""" + "" + r"""} \opening{Dear Hiring Manager,} """ + latex_code + r"""

            \closing{Sincerely,}

            \end{letter}

            \end{document}

            """
            create_pdf_from_latex(latex_codes, "cover_letter")
            st.success('PDF generated successfully! And you can check the main cover_letter.pdf file in the root folder.')
            st.text_area("Cover Letter:", value=latex_code, height=800)
        except Exception as e:
            st.error(f"An error occurred: {e}")
    else:
        st.error('Please paste a job description.')
```

## OpenAPI Specs
--------------

```yml
openapi: 3.1.0
info:
  version: 1.0.0
  title: Example API
  termsOfService: https://example.com/terms/
  contact:
    name: Contact our support
    email: contact@example.com
    url: http://example.com/contact
  license:
    name: Apache 2.0
    url: http://www.apache.org/licenses/LICENSE-2.0.html
  x-logo:
    url: https://redocly.github.io/openapi-template/logo.png
    altText: OpenAPI example logo
  description: This is an **example** API to demonstrate features of the OpenAPI specification.
tags:
  - name: Echo
    description: Example echo operations.
  - name: User
    description: Example actions on user accounts.
  - name: Admin
    description: Example operations reserved for administrators.
  - name: Info
    description: Example operations for retrieving information.
  - name: Tag
    description: This is a tag description.
x-tagGroups:
  - name: General
    tags:
      - User
      - Info
      - Echo
  - name: Administration
    tags:
      - Admin
servers:
  - url: https://{tenant}/api/v1
    variables:
      tenant:
        default: www
        description: Your tenant id
  - url: https://example.com/api/v1
paths:
  /users/{username}:
    $ref: paths/users_{username}.yaml
  /user:
    $ref: paths/user.yaml
  /user/list:
    $ref: paths/user-status.yaml
  /pathItem:
    $ref: paths/pathItem.yaml
  /pathItemWithExamples:
    $ref: paths/pathItemWithExamples.yaml
  /echo:
    $ref: paths/echo.yaml
components:
  securitySchemes:
    main_auth:
      description: Example description text of the OAuth2 scheme.
      type: oauth2
      flows:
        implicit:
          authorizationUrl: http://example.com/api/oauth/dialog
          scopes:
            read:users: read user info
            write:users: modify or remove users
    api_key:
      description: Example description text of the API key scheme.
      type: apiKey
      in: header
      name: api_key
    basic_auth:
      type: http
      scheme: basic
webhooks:
  userInfo:
    post:
      summary: New user webhook
      description: Information about a new user in the system.
      operationId: userInfo
      tags:
        - Info
      requestBody:
        content:
          application/json:
            schema:
              $ref: components/schemas/User.yaml
      responses:
        200:
          description: Successfully retrieved information about a new user.
      security:
        - api_key: []
```